In [ ]:
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from wtforms.widgets import TextArea
import datetime
import pandas as pd
import csv

app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'thisisasecretkey'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))


class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    bio = db.Column(db.Text)
    admin = db.Column(db.Boolean)
    discussion = db.relationship('Discussion', backref='writer', lazy='dynamic')


class Discussion(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    discussion_body = db.Column(db.String(100))
    discussion_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    author = db.Column(db.String(255))
    date_created = db.Column(db.TIMESTAMP, default=datetime.datetime.utcnow, nullable=False)


class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")
    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")

class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")


class NewDiscussionForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(min=1)], render_kw={"placeholder": "Title"})
    discussion_body = StringField(u'Text', widget=TextArea())
    submit = SubmitField("Add Opinion")

class UpdateDiscussionForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(min=1)], render_kw={"placeholder": "Title"})
    discussion_body = StringField(u'Text', widget=TextArea())
    submit = SubmitField("Update Discussion")

@app.route('/home')
def home():
    return render_template('index.html', title='Home')

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/drama")
def drama():
    df = pd.read_csv("dramas.csv")
    list = df.to_dict('records')
    return render_template('drama.html', entries = list)

@app.route("/detail1")
def detail1():
    return render_template('detail1.html')

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for('my_discussion'))
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password,bio='',admin=0)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)


@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))


@app.route('/discussion', methods=['GET','POST'])
def view_discussions():
    discussions = Discussion.query.all()
    return render_template('discussion.html', discussions=discussions, title='Discussion')

@app.route('/new-discussion', methods=['GET','POST'])
@login_required
def new_discussion():
    form = NewDiscussionForm()
    if form.validate_on_submit():
        new_discussion = Discussion(title=form.title.data, discussion_body=form.discussion_body.data, writer=current_user, author=current_user.username)
        db.session.add(new_discussion)
        db.session.commit()
    return render_template('new_discussion.html', title='New Discussion', form=form)


@app.route('/profile', methods=['GET','POST'])
@login_required
def my_discussion():
    if current_user.admin:
        discussions = Discussion.query.all()
    else:
        discussions = Discussion.query.filter_by(writer=current_user).all()
    return render_template('profile.html', discussions=discussions, title='My Dicussion')

@app.route('/delete-discussion/<int:discussion_id>', methods=['GET',"POST"])
@login_required
def delete_discussion(discussion_id):
    discussion = Discussion.query.get_or_404(discussion_id)
    db.session.delete(discussion)
    db.session.commit()
    return redirect(url_for('my_discussion'))

@app.route('/update-discussion/<int:discussion_id>', methods=['GET',"POST"])
@login_required
def update_discussion(discussion_id):
    form = UpdateDiscussionForm()
    discussion = Discussion.query.get_or_404(discussion_id)
    if request.method == 'GET':
        form.title.data = discussion.title
        form.discussion_body.data = discussion.discussion_body
    if form.validate_on_submit():
            Discussion.query.filter_by(id=discussion_id).first()
            if discussion:
                discussion.title = form.title.data
                discussion.discussion_body = form.discussion_body.data
                db.session.commit()
                print("Discussion id=",discussion_id)
                print("Updated.")
                return redirect(url_for('my_discussion'))
    return render_template('update_discussion.html', form=form, title='Update Discussion', id=discussion_id)


@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

if __name__ == '__main__':
    app.run('localhost',2234)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


D:\Graduate School\COMM\5961\Anaconda\lib\site-packages\flask_sqlalchemy\__init__.py:851: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  warnings.warn(
D:\Graduate School\COMM\5961\Anaconda\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:2234/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Nov/2021 06:37:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 06:37:34] "GET /assets/mail/jqBootstrapValidation.js HTTP/1.1" 404 -
127.0.0.1 - - [15/Nov/2021 06:37:34] "GET /assets/mail/jqBootstrapValidation.js HTTP/1.1" 404 -
127.0.0.1 - - [15/Nov/2021 06:37:35] "GET /assets/mail/contact_me.js HTTP/1.1" 404 -
127.0.0.1 - - [15/Nov/2021 06:37:36] "GET /login HTTP/1